In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, col, lower, regexp_replace,concat

spark = SparkSession.builder \
.appName('dpe_existants_rawdata_dl_job') \
.master('spark://spark-master:7077') \
.config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
.config('spark.ui.port', '4041') \
.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/30 16:26:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [66]:
enedis_2023 = spark.read.csv('hdfs:///hadoop/dfs/data/enedis/raw_data/consommation-annuelle-residentielle_2023.csv',sep=';',header=True)

df = enedis_2023.withColumn("adresse_brute", concat_ws(" ",
    col("adresse"),
    col("code_commune"),
    col("nom_commune")
))
enedis_normaliser = df.withColumn("adresse_brute", lower(regexp_replace(col("adresse_brute"), r"\s+", " ")))

In [58]:
enedis_normaliser.select("adresse_brute").show(10)

+--------------------+
|       adresse_brute|
+--------------------+
|18 rue du chasnot...|
|15 rue du chasnot...|
|1 rue du languedo...|
|7 rue du lycee 25...|
|16 rue du lycee 2...|
|1 rue du lycee 25...|
|13 rue du muguet ...|
|4 rue du palais 2...|
|13 rue du palais ...|
|7 rue du petit ba...|
+--------------------+
only showing top 10 rows



In [11]:
ban = spark.read.csv('hdfs:///hadoop/dfs/data/base-adresse-national/',sep=';',header=True)


In [15]:
# Création d'une colonne adresse normalisée dans le DataFrame de la BAN
ban_normaliser = ban.withColumn("adresse_normalisee_insee", concat_ws(" ",
    col("numero"),
    col("nom_voie"),
    col("code_insee"),
    col("nom_commune")
))
ban_normaliser = ban_normaliser.withColumn("adresse_normalisee_insee", lower(regexp_replace(col("adresse_normalisee_insee"), r"\s+", " ")))

In [67]:
# Réaliser la jointure avec les conditions multiples
enedis_join_ban = enedis_normaliser.join(ban_normaliser.select("adresse_normalisee_insee", "id"), enedis_normaliser.adresse_brute == ban_normaliser.adresse_normalisee_insee, how="inner")
distinct_values = enedis_join_ban.select("adresse_brute").distinct()
distinct_values.show(truncate=False)

# Affichage du résultat
enedis_join_ban.show(1,truncate=False,vertical=True)

+------------------------------------------------------+
|adresse_brute                                         |
+------------------------------------------------------+
|296 avenue aristide briand 92007 bagneux              |
|6 avenue mary 92009 bois-colombes                     |
|3 rue philippe de metz 92009 bois-colombes            |
|5 rue gilbert rousset 92004 asnières-sur-seine        |
|134 avenue henri barbusse 92004 asnières-sur-seine    |
|4 rue maurice bokanowski 92004 asnières-sur-seine     |
|5 rue neuve des mourinoux 92004 asnières-sur-seine    |
|17 rue des iris 92002 antony                          |
|51 avenue raymond aron 92002 antony                   |
|16 avenue de la providence 92002 antony               |
|48 rue de garches 92076 vaucresson                    |
|104 rue sadi carnot 92075 vanves                      |
|26 avenue jean perrin 92071 sceaux                    |
|8 place henri iv 92073 suresnes                       |
|1 rue des bas rogers 92073 sur

-RECORD 0---------------------------------------------------------------------------------------------
 annee                                                   | 2023                                       
 code_iris                                               | 590090206                                  
 nom_iris                                                | Château                                    
 numero_de_voie                                          | 1                                          
 indice_de_repetition                                    | NULL                                       
 type_de_voie                                            | ALLEE                                      
 libelle_de_voie                                         | CHANTECLER                                 
 code_commune                                            | 59009                                      
 nom_commune                                             | Villeneuve-d'A

In [68]:
enedis_join_ban.count()

248893

In [69]:
# Réduire le DataFrame à une seule partition
single_partition_df = enedis_join_ban.coalesce(1)

# Écrire le DataFrame en un seul fichier CSV 
single_partition_df.write.mode('overwrite').format('csv') \
    .option("path", "hdfs:///hadoop/dfs/data/enedis/staging/consommation-annuelle-residentielle_2023.csv") \
    .option("sep", ";")  \
    .option("header", "true").save()   

In [70]:
a = spark.read.csv('hdfs:///hadoop/dfs/data/enedis/staging/consommation-annuelle-residentielle_2022.csv',sep=';',header=True)


In [71]:
a.printSchema()

root
 |-- annee: string (nullable = true)
 |-- code_iris: string (nullable = true)
 |-- nom_iris: string (nullable = true)
 |-- numero_de_voie: string (nullable = true)
 |-- indice_de_repetition: string (nullable = true)
 |-- type_de_voie: string (nullable = true)
 |-- libelle_de_voie: string (nullable = true)
 |-- code_commune: string (nullable = true)
 |-- nom_commune: string (nullable = true)
 |-- segment_de_client: string (nullable = true)
 |-- nombre_de_logements: string (nullable = true)
 |-- consommation_annuelle_totale_de_l_adresse_mwh: string (nullable = true)
 |-- consommation_annuelle_moyenne_par_site_de_l_adresse_mwh: string (nullable = true)
 |-- consommation_annuelle_moyenne_de_la_commune_mwh: string (nullable = true)
 |-- adresse: string (nullable = true)
 |-- code_epci: string (nullable = true)
 |-- code_departement: string (nullable = true)
 |-- code_region: string (nullable = true)
 |-- tri_des_adresses: string (nullable = true)
 |-- adresse_brute: string (nullable = 